# COMUNICACIONES DIGITALES
> Optativa GIEC y GIT<br>
> Curso 2021/22

## Práctica 1c - Codificación de audio

En esta práctica vamos a ver algunas ideas relacionadas con la codificación de fuente aplicada al caso particular de ficheros de audio.
Se verán dos casos distintos:
 - Codificación en el dominio del tiempo: aunque la codificación de audio nunca se hace en el dominio del tiempo, este ejemplo resulta útil para ver de forma sencilla el efecto de una codificación de fuente en este tipo de archivos.
  - Codificación en el dominio de la frecuencia: Este caso se parece más a las aplicaciones reales, y permite ver con algo más de detalle el funcionamiento de un codificador de audio real.
 

### 1 - GENERACIÓN DEL MENSAJE
Generamos un mensaje que será la entrada a nuestro sistema. Se puede probar, si se quiere, con mensajes más largos o incluso con otros tipos de archivos de entrada.

In [1]:
#Librerías necesarias
import scipy.io.wavfile as wav  # Para leer y escribir archivos .wav
import numpy as np              # Librería matemática
import scipy.fftpack            # Para las transformadas del coseno (DCT e IDCT)

###############
# CONFIGURACIÓN
###############
fichero_entrada = "entrada.wav"
fichero_salida_1 = "salida_tiempo.wav"
fichero_salida_2 = "salida_frecuencia.wav"
# Escalón de cuantificación para el dominio temporal
Q_t = 512
# Escalón de cuantificación para el dominio de la frecuencia
Q_f = 64
# Tamaño de cada trama para la transformada DCT
N = 2048

###############
# PASOS PREVIOS
###############
fs, mensaje = wav.read(fichero_entrada)


### 2 - CÁLCULOS PREVIOS
Antes de realizar la codificación, calculo el número de bits necesarios para codificar el mensaje tal cual, sin hacer nada, y cuál sería el mínimo teórico según el cálculo de la entropía.

In [2]:
KB = 0
print("Tamaño del archivo sin codificar: %.1f" % KB, "KBytes")

KB = 0
print("Tamaño mínimo que podríamos conseguir en teoría: %.1f" % KB, "Kbytes")


Tamaño del archivo sin codificar: 0.0 KBytes
Tamaño mínimo que podríamos conseguir en teoría: 0.0 Kbytes


### 3 - CODIFICACIÓN EN EL DOMINIO DEL TIEMPO
Comenzamos con la codificación Huffmann. 
En primer lugar calculo el diccionario adaptado al mensaje de entrada, y muestro el resultado. 
 - **CodFuente.gen_huffman_dic**: Genera dos diccionarios para la codificación Huffman: *probabilidades* con pares símbolo/probabilidad y *códigos* con pares símbolo/código binario.


In [3]:
##################################################
# PROCEDIMIENTO 1
# Trabajamos directamente en el dominio del tiempo
##################################################
# Cuantificación del mensaje
x_t = np.round(mensaje/Q_t)
# Codificación fuente (Huffman)
from CodFuente import gen_huffman_dic, huffman_cod
diccionario_huffman, codigos_huffman = gen_huffman_dic(x_t)
bits = huffman_cod(x_t, codigos_huffman)

KB = 0
print("Tamaño del archivo codificado en el tiempo: %.1f" % KB, "Kbytes")

# Decodificación fuente
from CodFuente import huffman_dec
bits_r = np.array(huffman_dec(bits, codigos_huffman))
# Cuantificación inversa
x_r = bits_r*Q_t
# Lo convertimos a formato entero 16bits y grabamos el archivo 
y = x_r.astype(np.int16)
wav.write(fichero_salida_1, fs, y)

¡OJO! HAY QUE IMPLEMENTAR LA FUNCIÓN huffman_cod
Tamaño del archivo codificado en el tiempo: 0.0 Kbytes


### 4 - CODIFICACIÓN EN EL DOMINIO DE LA FRECUENCIA
Utilizando el diccionario de antes, codifico el mensaje y calculo su tamaño una vez codificado. 

In [4]:
#######################################
# PROCEDIMIENTO 2
# Trabajamos en el dominio transformado
#######################################
# Antes de nada relleno con ceros para que el tamaño sea múltiplo de N
mensaje = np.append(mensaje, np.zeros(len(mensaje)%N))

# Voy cogiendo tramas de N muestras y calculo la DCT de cada una de ellas
x_t = np.array([])
for i in range(0,len(mensaje),N):
    trama = mensaje[i:i+N]
    trama_dct = scipy.fftpack.dct(trama, norm='ortho')    #Transformada DCT
    trama_dct = np.round(trama_dct/Q_f)       #Cuantificación
    x_t = np.append(x_t, trama_dct)

# Codificación fuente (Huffman)
from CodFuente import gen_huffman_dic, huffman_cod
diccionario_huffman, codigos_huffman = gen_huffman_dic(x_t)
bits = huffman_cod(x_t, codigos_huffman)

KB = 0
print("Tamaño del archivo codificado en frecuencia: %.1f" % KB, "KBytes")

#Decodificación fuente
from CodFuente import huffman_dec
bits_r = np.array(huffman_dec(bits, codigos_huffman))

#Ahora hacemos la transformada inversa. Voy cogiendo cada trama de N muestras y calculo la IDCT
y = np.array([])
for i in range(0, len(bits_r), N):
    trama = bits_r[i:i+N]
    trama = trama*Q_f                       #Cuantificación inversa
    trama_idct = scipy.fftpack.idct(trama, norm='ortho')  #Transformada IDCT
    y = np.append(y, trama_idct)

#Convierto todo a formato entero de 16 bits y grabo el archivo
y = y.astype(np.int16)
wav.write(fichero_salida_2, fs, y)

¡OJO! HAY QUE IMPLEMENTAR LA FUNCIÓN huffman_cod
Tamaño del archivo codificado en frecuencia: 0.0 KBytes
